# Multiple Agents (Group Chat) Demo

This notebook demonstrates how to interact with **multiple agents** simultaneously using the .NET Agent Framework backend API.

## What You'll Learn
- How to send messages to multiple agents at once
- How agents collaborate to answer questions
- Understanding group chat orchestration
- Viewing individual agent contributions
- Working with multi-agent responses

## Prerequisites
Make sure the backend API is running on `http://localhost:8000`

## Setup - Install Required Packages

In [2]:
#r "nuget: System.Net.Http.Json"
#r "nuget: System.Text.Json"

using System.Net.Http;
using System.Net.Http.Json;
using System.Text.Json;
using System.Text.Json.Serialization;

Installed Packages System.Net.Http.Json, 10.0.0 System.Text.Json, 10.0.0

## Configure API Connection

In [1]:
// API Configuration
var apiBaseUrl = "http://localhost:8000";
var httpClient = new HttpClient { BaseAddress = new Uri(apiBaseUrl) };
httpClient.Timeout = TimeSpan.FromMinutes(2);

Console.WriteLine($"✅ Connected to API: {apiBaseUrl}");

Error: (3,22): error CS0246: The type or namespace name 'HttpClient' could not be found (are you missing a using directive or an assembly reference?)

## Step 1: Get Available Agents

In [ ]:
// Get all available agents
var agentsResponse = await httpClient.GetFromJsonAsync<JsonElement>("/agents");

Console.WriteLine("📋 Available Agents for Group Chat:");
Console.WriteLine("===================================");

var agents = agentsResponse.GetProperty("agents");
var agentNames = new List<string>();

foreach (var agent in agents.EnumerateArray())
{
    var name = agent.GetProperty("name").GetString();
    var type = agent.GetProperty("type").GetString();
    agentNames.Add(name);
    
    Console.WriteLine($"🤖 {name} ({type})");
}

Console.WriteLine($"\n✅ Total agents available: {agentNames.Count}");

## Step 2: Send Message to Multiple Agents (User-Friendly Format)

When you specify multiple agents, the framework orchestrates them to work together on your question.

In [ ]:
// Prepare chat request for multiple agents
var multiAgentRequest = new
{
    message = "What are the latest trends in cloud computing and how do they impact enterprise IT?",
    agents = new[] { "azure_openai_agent", "ms_foundry_people_agent" },  // Multiple agents
    max_turns = 3,  // Allow up to 3 turns of conversation
    format = "user_friendly"  // Synthesized response
};

Console.WriteLine("📤 Sending message to MULTIPLE agents...");
Console.WriteLine($"Message: {multiAgentRequest.message}");
Console.WriteLine($"Agents: {string.Join(", ", multiAgentRequest.agents)}");
Console.WriteLine($"Max Turns: {multiAgentRequest.max_turns}\n");

// Send the request
var response = await httpClient.PostAsJsonAsync("/chat", multiAgentRequest);
response.EnsureSuccessStatusCode();

var result = await response.Content.ReadFromJsonAsync<JsonElement>();

// Display synthesized response
Console.WriteLine("\n💬 Synthesized Response:");
Console.WriteLine("========================");
Console.WriteLine($"Primary Agent: {result.GetProperty("agent").GetString()}");
Console.WriteLine($"Session ID: {result.GetProperty("session_id").GetString()}");
Console.WriteLine($"\nContent:\n{result.GetProperty("content").GetString()}");

// Display metadata
if (result.TryGetProperty("metadata", out var metadata))
{
    Console.WriteLine($"\n📊 Collaboration Metadata:");
    Console.WriteLine($"   Agent Count: {metadata.GetProperty("agent_count").GetInt32()}");
    Console.WriteLine($"   Total Turns: {metadata.GetProperty("total_turns").GetInt32()}");
    Console.WriteLine($"   Is Group Chat: {metadata.GetProperty("is_group_chat").GetBoolean()}");
    
    if (metadata.TryGetProperty("contributing_agents", out var contributors))
    {
        var contributorList = string.Join(", ", contributors.EnumerateArray().Select(c => c.GetString()));
        Console.WriteLine($"   Contributing Agents: {contributorList}");
    }
}

## Step 3: Send Message with Detailed Format

The detailed format shows each agent's individual contribution to the conversation.

In [ ]:
// Request with detailed format to see each agent's contribution
var detailedRequest = new
{
    message = "How can AI and machine learning improve customer service in retail?",
    agents = new[] { "azure_openai_agent", "bedrock_agent" },
    max_turns = 4,
    format = "detailed"  // Show individual agent responses
};

Console.WriteLine("📤 Sending message with DETAILED format...");
Console.WriteLine($"Message: {detailedRequest.message}");
Console.WriteLine($"Agents: {string.Join(", ", detailedRequest.agents)}\n");

var detailedResponse = await httpClient.PostAsJsonAsync("/chat", detailedRequest);
detailedResponse.EnsureSuccessStatusCode();

var detailedResult = await detailedResponse.Content.ReadFromJsonAsync<JsonElement>();

// Display conversation details
Console.WriteLine("\n💬 Detailed Group Chat Response:");
Console.WriteLine("=================================");
Console.WriteLine($"Conversation ID: {detailedResult.GetProperty("conversation_id").GetString()}");
Console.WriteLine($"Total Turns: {detailedResult.GetProperty("total_turns").GetInt32()}");

// Show active participants
var participants = detailedResult.GetProperty("active_participants");
Console.WriteLine($"\n👥 Active Participants ({participants.GetArrayLength()}):");
foreach (var participant in participants.EnumerateArray())
{
    Console.WriteLine($"   - {participant.GetString()}");
}

// Show individual agent responses
var responses = detailedResult.GetProperty("responses");
Console.WriteLine($"\n📝 Individual Agent Contributions ({responses.GetArrayLength()}):");
Console.WriteLine("=".PadRight(80, '='));

int turnNum = 1;
foreach (var resp in responses.EnumerateArray())
{
    var agentName = resp.GetProperty("agent").GetString();
    var content = resp.GetProperty("content").GetString();
    var meta = resp.GetProperty("metadata");
    var turn = meta.GetProperty("turn").GetInt32();
    
    Console.WriteLine($"\n[Turn {turn}] 🤖 {agentName}");
    Console.WriteLine("-".PadRight(80, '-'));
    Console.WriteLine(content);
    
    turnNum++;
}

// Show summary if available
if (detailedResult.TryGetProperty("summary", out var summary) && summary.ValueKind != JsonValueKind.Null)
{
    Console.WriteLine($"\n📋 Conversation Summary:");
    Console.WriteLine("=".PadRight(80, '='));
    Console.WriteLine(summary.GetString());
}

// Metadata
var meta2 = detailedResult.GetProperty("metadata");
Console.WriteLine($"\n📊 Group Chat Metadata:");
Console.WriteLine($"   Group Chat Type: {meta2.GetProperty("group_chat_type").GetString()}");
Console.WriteLine($"   Agent Count: {meta2.GetProperty("agent_count").GetInt32()}");
Console.WriteLine($"   Response Type: {meta2.GetProperty("response_type").GetString()}");

## Step 4: Complex Query with Multiple Agents

Try a more complex question that requires different types of expertise.

In [ ]:
// Complex query requiring multiple perspectives
var complexRequest = new
{
    message = @"I need help understanding: 
        1. What is Azure OpenAI Service?
        2. How can it be integrated with enterprise applications?
        3. What are the security considerations?",
    agents = new[] { "azure_openai_agent", "openai_agent" },
    max_turns = 5,
    format = "user_friendly"
};

Console.WriteLine("📤 Sending complex multi-part question...");
Console.WriteLine($"Agents: {string.Join(", ", complexRequest.agents)}\n");

var complexResponse = await httpClient.PostAsJsonAsync("/chat", complexRequest);
complexResponse.EnsureSuccessStatusCode();

var complexResult = await complexResponse.Content.ReadFromJsonAsync<JsonElement>();

Console.WriteLine("\n💬 Comprehensive Response:");
Console.WriteLine("==========================");
Console.WriteLine(complexResult.GetProperty("content").GetString());

// Show collaboration stats
var meta = complexResult.GetProperty("metadata");
Console.WriteLine($"\n📊 Collaboration Statistics:");
Console.WriteLine($"   Agents Participated: {meta.GetProperty("agent_count").GetInt32()}");
Console.WriteLine($"   Total Conversation Turns: {meta.GetProperty("total_turns").GetInt32()}");

if (meta.TryGetProperty("contributing_agents", out var contrib))
{
    Console.WriteLine($"   Contributors:");
    foreach (var c in contrib.EnumerateArray())
    {
        Console.WriteLine($"      - {c.GetString()}");
    }
}

## Step 5: Compare Single vs Multiple Agent Responses

Let's compare how a single agent vs multiple agents handle the same question.

In [ ]:
var testQuestion = "What are the benefits and challenges of implementing microservices architecture?";

Console.WriteLine($"Test Question: {testQuestion}");
Console.WriteLine("\n" + "=".PadRight(80, '='));

// Test 1: Single Agent
Console.WriteLine("\n🔵 Test 1: SINGLE AGENT Response");
Console.WriteLine("-".PadRight(80, '-'));

var singleAgentReq = new
{
    message = testQuestion,
    agents = new[] { "azure_openai_agent" },
    format = "user_friendly"
};

var singleResp = await httpClient.PostAsJsonAsync("/chat", singleAgentReq);
var singleResult = await singleResp.Content.ReadFromJsonAsync<JsonElement>();

Console.WriteLine($"Agent: {singleResult.GetProperty("agent").GetString()}");
var singleContent = singleResult.GetProperty("content").GetString();
Console.WriteLine($"Response Length: {singleContent.Length} characters");
Console.WriteLine($"\n{singleContent.Substring(0, Math.Min(300, singleContent.Length))}...");

// Test 2: Multiple Agents
Console.WriteLine("\n\n🟢 Test 2: MULTIPLE AGENTS Response");
Console.WriteLine("-".PadRight(80, '-'));

var multiAgentReq = new
{
    message = testQuestion,
    agents = new[] { "azure_openai_agent", "openai_agent" },
    max_turns = 3,
    format = "user_friendly"
};

var multiResp = await httpClient.PostAsJsonAsync("/chat", multiAgentReq);
var multiResult = await multiResp.Content.ReadFromJsonAsync<JsonElement>();

var multiMeta = multiResult.GetProperty("metadata");
Console.WriteLine($"Agents Involved: {multiMeta.GetProperty("agent_count").GetInt32()}");
Console.WriteLine($"Total Turns: {multiMeta.GetProperty("total_turns").GetInt32()}");

var multiContent = multiResult.GetProperty("content").GetString();
Console.WriteLine($"Response Length: {multiContent.Length} characters");
Console.WriteLine($"\n{multiContent.Substring(0, Math.Min(300, multiContent.Length))}...");

Console.WriteLine("\n\n📊 Comparison Summary:");
Console.WriteLine("=".PadRight(80, '='));
Console.WriteLine($"Single Agent: {singleContent.Length} chars, 1 agent");
Console.WriteLine($"Multiple Agents: {multiContent.Length} chars, {multiMeta.GetProperty("agent_count").GetInt32()} agents, {multiMeta.GetProperty("total_turns").GetInt32()} turns");
Console.WriteLine($"\n💡 Multiple agents can provide more comprehensive responses through collaboration!");

## Step 6: Get Available Group Chat Templates

The API provides pre-configured templates for common multi-agent scenarios.

In [ ]:
// Get available templates
try
{
    var templatesResponse = await httpClient.GetFromJsonAsync<JsonElement>("/chat/templates");
    
    Console.WriteLine("📋 Available Group Chat Templates:");
    Console.WriteLine("==================================");
    
    var templates = templatesResponse.GetProperty("templates");
    
    foreach (var template in templates.EnumerateArray())
    {
        var name = template.GetProperty("name").GetString();
        var description = template.GetProperty("description").GetString();
        var agentCount = template.GetProperty("agent_count").GetInt32();
        
        Console.WriteLine($"\n📝 Template: {name}");
        Console.WriteLine($"   Description: {description}");
        Console.WriteLine($"   Agents: {agentCount}");
        
        if (template.TryGetProperty("agents", out var templateAgents))
        {
            var agentList = string.Join(", ", templateAgents.EnumerateArray().Select(a => a.GetString()));
            Console.WriteLine($"   Agent Names: {agentList}");
        }
    }
}
catch (Exception ex)
{
    Console.WriteLine($"⚠️  Templates endpoint not available: {ex.Message}");
}

## Summary

In this notebook, you learned:
- ✅ How to send messages to multiple agents simultaneously
- ✅ Understanding group chat orchestration and collaboration
- ✅ Viewing individual agent contributions vs synthesized responses
- ✅ Comparing single-agent vs multi-agent responses
- ✅ Using detailed format to see the conversation flow
- ✅ Exploring pre-configured group chat templates

## Available Agents for Multi-Agent Scenarios

### 1. Azure OpenAI Agent (`azure_openai_agent`)
- **Provider**: azure_openai
- **Type**: azure_openai_agent
- **Best For**: General-purpose tasks, analysis, problem-solving

### 2. Microsoft Foundry People Agent (`ms_foundry_people_agent`)
- **Provider**: ms_foundry
- **Type**: ms_foundry_agent
- **Best For**: People search, contact discovery, team coordination

### 3. Bedrock HR Agent (`bedrock_agent`)
- **Provider**: aws
- **Type**: bedrock_agent
- **Best For**: HR policies, benefits, workplace guidance

### 4. OpenAI Agent (`openai_agent`)
- **Provider**: openai
- **Type**: openai_agent
- **Best For**: Development help, coding, technical analysis

## Key Takeaways
- Multiple agents can provide more comprehensive and nuanced responses
- The framework automatically orchestrates agent collaboration
- You can choose between synthesized (user-friendly) or detailed responses
- Templates provide pre-configured multi-agent scenarios
- Different agent combinations work well for different types of questions

## Memory Feature (NEW)
You can now enable long-running memory across all agent types:
- Add `enable_memory: true` to any chat request (single or multi-agent)
- Agents will extract and remember your name and persona
- See **04-LongRunningMemory-Demo.ipynb** for complete examples

**Example with Memory:**
```json
{
  "message": "My name is Alice and I'm a data scientist",
  "agents": ["azure_openai_agent", "ms_foundry_people_agent"],
  "enable_memory": true,
  "max_turns": 3
}
```

## Next Steps
- Try the **03-ContentSafety-Demo** to see how content is moderated
- Check out the **04-LongRunningMemory-Demo** for memory and session management
- Explore the **01-SingleAgent-Demo** for basic agent interaction